
# Taller 1: Modelos de Recomendación y Aplicación de Perceptrón y Redes Neuronales ADALINE

Este notebook implementa un sistema de recomendación utilizando datos públicos de videojuegos de Steam, además de técnicas de escalado y recomendaciones basadas en el comportamiento de los usuarios.

## Objetivo:
- Construir un sistema de recomendación usando usuarios, videojuegos y tiempo de juego.


In [11]:

import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD, KNNBasic
from surprise import accuracy
from surprise.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity



## Carga de Datos:
Cargamos los datos del archivo CSV de Steam, donde tenemos información sobre los usuarios, los videojuegos, el comportamiento del usuario (si juega o posee el juego), y las horas de juego.


In [12]:

# Cargar los datos
df = pd.read_csv('Bases/steam-200k.csv', names=['ID', 'Juego', 'Comportamiento', 'Horas', '0'], index_col=0)

# Eliminar la columna no necesaria
base = df.drop(columns=['0'])

# Filtrar solo los registros donde el comportamiento es 'play'
data = base.reset_index()
data = data[data['Comportamiento']=="play"]

# Mostrar los primeros datos
data.head()


,ID,Juego,Comportamiento,Horas
1,151603712,The Elder Scrolls V Skyrim,play,273.0
3,151603712,Fallout 4,play,87.0
5,151603712,Spore,play,14.9
7,151603712,Fallout New Vegas,play,12.1
9,151603712,Left 4 Dead 2,play,8.9



## Selección de Variables y Preprocesamiento:
Seleccionamos las columnas relevantes: ID del usuario, el nombre del videojuego y las horas jugadas. Además, aplicamos un escalado a las horas de juego para normalizarlas.


In [13]:

# Seleccionar las columnas de interés
data = data[['ID', 'Juego', 'Horas']]

# Aplicar escalado a la columna de horas
scaler = MinMaxScaler()
data['Horas_escaladas'] = scaler.fit_transform(data[['Horas']])


dataf = data.copy()
# Mostrar los datos procesados
data.head()


,ID,Juego,Horas,Horas_escaladas
1,151603712,The Elder Scrolls V Skyrim,273.0,0.023218
3,151603712,Fallout 4,87.0,0.007393
5,151603712,Spore,14.9,0.001259
7,151603712,Fallout New Vegas,12.1,0.001021
9,151603712,Left 4 Dead 2,8.9,0.000749



## Construcción del Sistema de Recomendación:
Utilizamos la biblioteca `surprise` para implementar un modelo de recomendación basado en un algoritmo de descomposición matricial SVD.


In [14]:

# Definir el lector de datos para Surprise
reader = Reader(rating_scale=(0, data['Horas'].max()))

# Convertir los datos para Surprise
datos_surprise = Dataset.load_from_df(data[['ID', 'Juego', 'Horas_escaladas']], reader)

# Dividir en conjunto de entrenamiento y prueba
entrenamiento, testset = train_test_split(datos_surprise, test_size=0.25)

# Crear el modelo SVD
model= KNNBasic()

# Entrenar el modelo
model.fit(entrenamiento)

# Realizar predicciones
predicciones = model.test(testset)

# Evaluar el modelo
accuracy.rmse(predicciones)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0183


0.018337090918201487


## Evaluación del Sistema de Recomendación:
Finalmente, evaluamos el rendimiento del modelo utilizando validación cruzada para calcular métricas como RMSE (Error Cuadrático Medio) y MAE (Error Absoluto Medio).


In [15]:
model.sim_options
predictions = model.test(testset)
accuracy.rmse(predictions)
model.predict(568, 519)
dataf.describe()

RMSE: 0.0183


,ID,Horas,Horas_escaladas
count,7.048900e+04,70489.000000,70489.000000
mean,1.058812e+08,48.878063,0.004150
std,7.150365e+07,229.335236,0.019511
min,5.250000e+03,0.100000,0.000000
25%,4.934256e+07,1.000000,0.000077
50%,9.169067e+07,4.500000,0.000374
75%,1.556738e+08,19.100000,0.001616
max,3.099031e+08,11754.000000,1.000000
